In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import random

In [2]:
def FrankeFunction(x,y):
    term1 = 0.75*np.exp(-(0.25*(9*x-2)**2) - 0.25*((9*y-2)**2))
    term2 = 0.75*np.exp(-((9*x+1)**2)/49.0 - 0.1*(9*y+1))
    term3 = 0.5*np.exp(-(9*x-7)**2/4.0 - 0.25*((9*y-3)**2))
    term4 = -0.2*np.exp(-(9*x-4)**2 - (9*y-7)**2)
    return term1 + term2 + term3 + term4 + np.random.normal(0, 0.1, x.shape)

In [3]:
def R2(y_data, y_model):
    return 1 - np.sum((y_data - y_model)**2) / np.sum((y_data - np.mean(y_data)) ** 2)

def MSE(y_data,y_model):
    n = np.size(y_model)
    return np.sum((y_data-y_model)**2)/n

In [4]:
def create_X(x, y, n):
    if len(x.shape) > 1:
        x = np.ravel(x) # flattens the matrices
        y = np.ravel(y)

    N = len(x) #number of x-variables, datapoints
    l = int((n+1)*(n+2)/2)     # Number of elements in beta - parameters, features
    X = np.ones((N,l)) #Making a matrix of dimentions given by the number of variables and number of parameters

    for i in range(1,n+1):
        q = int((i)*(i+1)/2)
        for k in range(i+1):
            X[:,q+k] = (x**(i-k))*(y**k)

    return X

In [8]:
def find_beta(X, z): 
    XT = X.T
    XTXinv = np.linalg.pinv(np.matmul(XT, X))
    XTz = np.matmul(XT, z)
    beta = np.matmul(XTXinv, XTz)

    return beta

In [9]:
size = 2000
noise = 0.05 # Level of noise
x = np.arange(0, 1, 1/size)
y = np.arange(0, 1, 1/size)
#x, y = np.meshgrid(x,y)

z = FrankeFunction(x, y)
z += (np.random.randn(size)*noise) #Added noise
print(len(x), len(y), len(z))

2000 2000 2000


In [10]:
from sklearn.utils import resample
def bootstrap(x, z, x_test, z_test, iterations = 100):
    MSEs = np.zeros(iterations) 
    R2s = np.zeros(iterations) 
    z_preds= []
    for i in range(iterations):
        bt_x, bt_z = resample(x, z)
        beta = find_beta(bt_x, bt_z) #Finding beta with new x train and z train
        z_pred = x_test @ beta #predict z with x_test
        z_preds.append(z_pred)
        mse = MSE(z_test, z_pred)
        r2 = R2(z_test, z_pred) # getting statistics of prediction in current bootstrap
        MSEs[i] = mse
        R2s[i] = r2
    
    zpreds = np.mean(z_preds)
    bt_err = np.mean( np.mean((z_test - z_preds)**2, axis=1, keepdims=True))
    bt_bias = np.mean((z_test - np.mean(z_preds, axis=1, keepdims=True))**2)
    bt_var = np.mean( np.var(z_preds) )
    boot_MSE = np.mean(MSEs)
    boot_R2 = np.mean(R2s)
    
    return boot_MSE, boot_R2, bt_err, bt_bias, bt_var

In [11]:
maxdegree = 20
scores_OLS_boot = np.zeros((maxdegree, 2))
degrees = np.linspace(1, maxdegree, maxdegree, dtype=int)
metrics = {'degree': degrees,'error': [], 'bias': [], 'variance': []}

for degree in degrees:
    X = create_X(x, y, degree)
    X_train, X_test, z_train, z_test = train_test_split(X,z,test_size=1/4)
    
    boot_n = 50

    bt_MSE, bt_R2, error, bias, var = bootstrap(X_train,z_train,X_test, z_test, iterations = boot_n) #bootstrapping the z-values to get a resampled set of the 'observed' data
    metrics['error'].append(error)
    metrics['bias'].append(bias)
    metrics['variance'].append(var)
    

    scores_OLS_boot[degree-1, 0] = bt_MSE
    scores_OLS_boot[degree-1, 1] = bt_R2

    
estimates = pd.DataFrame(scores_OLS_boot, columns=['MSE', 'R2'])
bt_results = pd.concat([pd.DataFrame(metrics), estimates], axis = 1)
bt_results = bt_results.set_index('degree')
display(bt_results)

degree7 = bt_results.loc[7]
print('    error              bias                 variance        ')
print('{} >= {} + {} = {}'.format(degree7['error'], degree7['bias'], degree7['variance'], degree7['bias']+ degree7['variance']))

,error,bias,variance,MSE,R2
degree,,,,,
1,0.042078,0.188838,0.136786,0.042078,0.777123
2,0.044492,0.195191,0.145637,0.044492,0.771994
3,0.023608,0.176688,0.166254,0.023608,0.866375
4,0.016388,0.190349,0.173921,0.016388,0.913900
5,0.018586,0.188231,0.169030,0.018586,0.901235
6,0.013486,0.190309,0.168344,0.013486,0.929133
7,0.012696,0.183244,0.167393,0.012696,0.930714
8,0.013543,0.186423,0.180134,0.013543,0.927352
9,0.013560,0.190347,0.177427,0.013560,0.928751


    error              bias                 variance        
0.012695638496939608 >= 0.1832443726472326 + 0.1673933600336152 = 0.3506377326808478
